###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2017 L.A. Barba, N.C. Clementi

# Life expectancy and wealth

Welcome to **Lesson 4** of the second module in _Engineering Computations_. This module gives you hands-on data analysis experience with Python, using real-life applications. The first three lessons provide a foundation in data analysis using a computational approach. They are:

1. [Lesson 1](http://go.gwu.edu/engcomp2lesson1): Cheers! Stats with beers.
2. [Lesson 2](http://go.gwu.edu/engcomp2lesson2): Seeing stats in a new light.
3. [Lesson 3](http://go.gwu.edu/engcomp2lesson3): Lead in lipstick.

You learned to do exploratory data analysis with data in the form of arrays: NumPy has built-in functions for many descriptive statistics, making it easy! And you also learned to make data visualizations that are both good-looking and effective in communicating and getting insights from data.

But NumPy can't do everything. So we introduced you to `pandas`, a Python library written _especially_ for data analysis. It offers a very powerful new data type: the _DataFrame_—you can think of it as a spreadsheet, conveniently stored in one Python variable. 

In this lesson, you'll dive deeper into `pandas`, using data for life expectancy and per-capita income over time, across the world.

## The best stats you've ever seen

[Hans Rosling](https://en.wikipedia.org/wiki/Hans_Rosling) was a professor of international health in Sweeden, until his death in Februrary of this year. He came to fame with the thrilling TED Talk he gave in 2006: ["The best stats you've ever seen"](https://www.ted.com/talks/hans_rosling_shows_the_best_stats_you_ve_ever_seen) (also on [YouTube](https://youtu.be/RUwS1uAdUcI), with ads). We highly recommend that you watch it! 

In that first TED Talk, and in many other talks and even a BBC documentary (see the [trailer](https://youtu.be/jbkSRLYSojo) on YouTube), Rosling uses data visualizations to tell stories about the world's health, wealth, inequality and development. Using software, he and his team created amazing animated graphics with data from the United Nations and World Bank.

According to a [blog post](https://www.gatesnotes.com/About-Bill-Gates/Remembering-Hans-Rosling) by Bill and Melinda Gates after Prof. Rosling's death, his message was simple: _"that the world is making progress, and that policy decisions should be grounded in data."_

In this lesson, we'll use data about life expectancy and per-capita income (in terms of the gross domestic product, GDP) around the world. Visualizing and analyzing the data will be our gateway to learning more about the world we live in.

Let's begin! As always, we start by importing the Python libraries for data analysis (and setting some plot parameters).

In [ ]:
import numpy
import pandas
from matplotlib import pyplot
%matplotlib inline

#Import rcParams to set font styles
from matplotlib import rcParams

#Set font style and size 
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

## Load and inspect the data

We found a website called [The Python Graph Gallery](https://python-graph-gallery.com), which has a lot of data visualization examples. 
Among them is a [Gapminder Animation](https://python-graph-gallery.com/341-python-gapminder-animation/), an animated GIF of bubble charts in the style of Hans Rosling. 
We're not going to repeat the same example, but we do get some ideas from it and re-use their data set. 
The data file is hosted on their website, and we can read it directly from there into a `pandas` dataframe, using the URL.

In [ ]:
# Read a dataset for life expectancy from a CSV file hosted online
url = 'https://python-graph-gallery.com/wp-content/uploads/gapminderData.csv'
life_expect = pandas.read_csv(url)

The first thing to do always is to take a peek at the data. 
Using the `shape` attribute of the dataframe, we find out how many rows and columns it has. In this case, it's kind of big to print it all out, so to save space we'll print a small portion of `life_expect`.
You can use a slice to do this, or you can use the [`DataFrame.head()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html) method, which returns by default the first 5 rows.

In [ ]:
life_expect.shape

In [ ]:
life_expect.head()

You can see that the columns hold six types of data: the country, the year, the population, the continent, the life expectancy, and the per-capita gross domestic product (GDP). 
Rows are indexed from 0, and the columns each have a **label** (also called an index). Using labels to access data is one of the most powerful features of `pandas`.

In the first five rows, we see that the country repeats (Afghanistan), while the year jumps by five. We guess that the data is arranged in blocks of rows for each country.

We can get a useful summary of the dataframe with the [`DataFrame.info()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.info.html) method: it tells us the number of rows and the number of columns (matching the output of the `shape` attribute) and then for each column, it tells us the number of rows that are populated (have non-null entries) and the type of the entries; finally it gives a breakdown of the types of data and an estimate of the memory used by the dataframe.

In [ ]:
life_expect.info()

The dataframe has 1704 rows, and every column has 1704 non-null entries, so there is no missing data. Let's find out how many entries of the same year appear in the data. 
In [Lesson 1](http://go.gwu.edu/engcomp2lesson1) of this module, you already learned to extract a column from a data frame, and use the [`series.value_counts()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html) method to answer our question.

In [ ]:
life_expect['year'].value_counts()

We have an even 142 occurrences of each year in the dataframe. The distinct entries must correspond to each country. It also is clear that we have data every five years, starting 1952 and ending 2007. We think we have a pretty clear picture of what is contained in this data set. What next?

## Grouping data for analysis

We have a dataframe with a `country` column, where countries repeat in blocks of rows, and a `year` column, where sets of 12 years (increasing by 5) repeat for every country. Tabled data commonly has this interleaved structure. And data analysis often involves grouping the data in various ways, to transform it, compute statistics, and visualize it.

With the life expectancy data, it's natural to want to analyze it by year (and look at geographical differences), and by country (and look at historical differences). 

In [Lesson 2](http://go.gwu.edu/engcomp2lesson2) of this module, we already learned how useful it was to group the beer data by style, and calculate means within each style. Let's get better acquainted with the powerful `groupby()` method for dataframes. First, grouping by the values in the `year` column:

In [ ]:
by_year = life_expect.groupby('year')

In [ ]:
type(by_year)

Notice that the type of the new variable `by_year` is different: it's a _GroupBy_ object, which—without making a copy of the data—is able to apply operations on each of the groups.

The [`GroupBy.first()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.first.html) method, for example, returns the first row in each group—applied to our grouping `by_year`, it shows the list of years (as a label), with the first country that appears in each year-group.

In [ ]:
by_year.first()

All the year-groups have the same first country, Afghanistan, so what we see is the population, life expectancy and per-capita income in Afghanistan for all the available years.
Let's save that into a new dataframe, and make a line plot of the population and life expectancy over the years.

In [ ]:
Afghanistan = by_year.first()

In [ ]:
Afghanistan['pop'].plot(figsize=(8,4),
                       title='Population of Afghanistan');

In [ ]:
Afghanistan['lifeExp'].plot(figsize=(8,4),
                       title='Life expectancy of Afghanistan');

Do you notice something interesting? It's curious to see that the population of Afghanistan took a fall after 1977. We have data every 5 years, so we don't know exactly when this fall began, but it's not hard to find the answer online. The USSR invaded Afghanistan in 1979, starting a conflict that lasted 9 years and resulted in an estimated death toll of one million civilians and 100,000 fighters [1]. Millions fled the war to neighboring countries, which may explain why we se a dip in population, but not a dip in life expectancy.

We can also get some descriptive statistics in one go with the [`DataFrame.describe()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html) method of `pandas`.

In [ ]:
Afghanistan.describe()

Let's now group our data by country, and use the `GroupBy.first()` method again to get the first row of each group-by-country. We know that the first year for which we have data is 1952, so let's immediately save that into a new variable named `year1952`, and keep playing with it. Below, we double-check the type of `year1952`, print the first five rows using the `head()` method, and get the minimum value of the population column.

In [ ]:
by_country = life_expect.groupby('country')

The first year for all groups-by-country is 1952. Let's save that first group into a new dataframe, and keep playing with it.

In [ ]:
year1952 = by_country.first()

In [ ]:
type(year1952)

In [ ]:
year1952.head()

In [ ]:
year1952['pop'].min()

## Visualizing the data

In [Lesson 3](http://go.gwu.edu/engcomp2lesson3) of this module, you learned to make bubble charts, allowing you to show at least three features of the data in one plot. We'd like to make a bubble chart of life expectancy vs. per-capita GDP, with the size of the bubble proportional to the population. To do that, we'll need to extract the population values into a NumPy array.

In [ ]:
populations = year1952['pop'].values

If you use the `populations` array unmodified as the size of the bubbles, they come out _huge_ and you get one solid color covering the figure (we tried it!). To make the bubble sizes reasonable, we divide by 60,000—an approximation to the minimum population—so the smallest bubble size is about 1 pt. Finally, we choose a logarithmic scale in the absissa (the GDP). Check it out!

In [ ]:
year1952.plot.scatter(figsize=(12,8), 
                       x='gdpPercap', y='lifeExp', s=populations/60000, 
                       title='Life expectancy in the year 1952',
                       edgecolors="white")
pyplot.xscale('log');

That's neat! But the Rosling bubble charts include one more feature in the data: the continent of each country, using a color scheme. Can we do that?

Matplotlib [colormaps](https://matplotlib.org/examples/color/colormaps_reference.html) offer several options for _qualitative_ data, using discrete colors mapped to a sequence of numbers. We'd like to use the `Accent` colormap to code countries by continent. But we need a numeric code to assign to each continent, so it can be mapped to a color.

The [Gapminder Animation](https://python-graph-gallery.com/341-python-gapminder-animation/) example at The Python Graph Gallery has a really good tip: use the `pandas` _Categorical_ data type, which gives you a numerical value for each category in a column containing qualitative (categorical) data. Let's see what we get if we apply  `pandas.Categorical()` to the `continent` column:

In [ ]:
pandas.Categorical(year1952['continent'])

We learn that the `continent` column has repeated entries of 5 distinct categories, one for each continent. Applying [`pandas.Categorical()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Categorical.html), we'll get an integer value—the _code_ of the category—associated to each entry. This is what we need to assign values in a colormap: we extract the `codes` attribute of the _Categorical_ data and save that into a new variable named `colors` (a NumPy array).

In [ ]:
colors = pandas.Categorical(year1952['continent']).codes

In [ ]:
type(colors)

In [ ]:
len(colors)

Now we're ready to re-do our bubble chart, using the array `colors` to set the color of the bubble (according to the continent for the given country).

In [ ]:
year1952.plot.scatter(figsize=(12,8), 
                         x='gdpPercap', y='lifeExp', s=populations/60000, 
                         c=colors, cmap='Accent',
                         title='Life expectancy vs per-capita GDP in the year 1952',
                         logx = 'True',
                         ylim = (25,85),
                         xlim = (1e2, 1e5),
                         edgecolors="white",
                         alpha=0.6);

##### Note:

We encountered a bug in `pandas` scatter plots! The labels of the $x$-axis disappeared when we added the colors to the bubbles. We tried several things to fix it, like adding the line `pyplot.xlabel("GDP per Capita")` at the end of the cell, but nothing worked. Searching online, we found an open [issue report](https://github.com/pandas-dev/pandas/issues/10611) for this problem.


##### Discuss with your neighbor:

What do you see in the colored bubble chart, in regards to 1952 conditions in different countries and different continents?

### Spaghetti plot of life expectancy

The bubble plot shows us that 1952 life expectancies varied quite a lot from country to country: from a minimum of under 30 years, to a maximum under 75 years. The first part of Prof. Rosling's dying message is _"that the world is making progress_. Is it the case that countries around the world _all_ make progress in life expectancy over the years?

We have an idea: what if we plot a line of life expectancy over time, for every country in the data set? It could be a bit messy, but it may give an _overall view_ of the world-wide progress in life expectancy.

Below, we'll make such a plot, with 142 lines: one for each country. This type of graphic is called a **spaghetti plot** …for obvious reasons!

To add a line for each country on the same plot, we'll use a `for`-statement and the `by_country` groups. For each country-group, the line plot takes the series `year` and `lifeExp` as $(x,y)$ coordinates. Since the spaghetti plot is quite busy, we also took off the box around the plot. Study this code carefully.

In [ ]:
pyplot.figure(figsize=(12,8))

for key,group in by_country:
    pyplot.plot(group['year'], group['lifeExp'], alpha=0.4)
    
pyplot.title('Life expectancy in the years 1952–2007, across 142 countries')
pyplot.box(on=None);

## Dig deeper and get insights from the data

The spaghetti plot shows a general upwards tendency, but clearly not all countries have a monotonically increasing life expectancy. Some show a one-year sharp drop (but remember, this data jumps every 5 years), while others drop over several years.
And something catastrophic happened to one country in 1977, and to another country in 1992.
Let's investigate this!

We'd like to explore the data for a particular year: first 1977, then 1992. For those years, we can get the minimum life expectancy, and then find out which country experienced it. 

To access a particular group in _GroupBy_ data, `pandas` has a `get_group(key)` method, where `key` is the label of the group.
For example, we can access yearly data from the `by_year` groups using the year as key. The return type will be a dataframe, containing the same columns as the original data.

In [ ]:
type(by_year.get_group(1977))

In [ ]:
type(by_year['lifeExp'].get_group(1977))

Now we can find the minimum value of life expectancy at the specific years of interest, using the `Series.min()` method. Let' do this for 1977 and 1992, and save the values in new Python variables, to reuse later.

In [ ]:
min_lifeExp1977 = by_year['lifeExp'].get_group(1977).min()
min_lifeExp1977

In [ ]:
min_lifeExp1992 = by_year['lifeExp'].get_group(1992).min()
min_lifeExp1992

Those values of life expectancy are just terrible! Are you curious to know what countries experienced the dramatic drops in life expectancy?

We can find the row _index_ of the minimum value, thanks to the [`pandas.Series.idxmin()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.idxmin.html) method. The row indices are preserved from the original dataframe `life_expect` to its groupings, so the index will help us identify the country. Check it out.

In [ ]:
by_year['lifeExp'].get_group(1977).idxmin()

In [ ]:
life_expect['country'][221]

In [ ]:
by_country.get_group('Cambodia')

We searched online to learn what was happening in Cambodia to cause such a drop in life expectancy in the 1970s. Indeed, Cambodia experienced a _mortality crisis_ due to several factors that combined into a perfect storm: war, ethnic cleansing and migration, collapse of the health system, and cruel famine [2].
It's hard for a country to keep vital statistics under such circumstances, and certainly there are uncertainties in the data for Cambodia in the 1970s.
However, various sources report a life expectancy there in 1977 that was _under 20 years_.
See, for example, the World Bank's interactive web page on [Cambodia](https://data.worldbank.org/country/cambodia).

There is something strange with the data from the The Python Graph Gallery. Is it wrong?
Maybe they are giving us _average_ life expectancy in a five-year period.
Let's look at the other dip in life expectancy, in 1992.

In [ ]:
by_year['lifeExp'].get_group(1992).idxmin()

In [ ]:
life_expect['country'][1292]

In [ ]:
by_country.get_group('Rwanda')

The World Bank's interactive web page on [Rwanda](https://data.worldbank.org/country/rwanda) gives a life expectancy of 28.1 in 1992, and even lower in 1993, at 27.6 years. 
This doesn't match the value from the data set we sourced from The Python Graph Gallery, which gives 23.6—and since this value is _lower_ than the minimum value given by the World Bank, we conclude that the discepancy is not caused by 5-year averaging.

## Checking data quality

All our work here started with loading a data set we found online. What if this data set has _quality_ problems? 

Well, nothing better than asking the author of the web source for the data. We used Twitter to communicate with the author of The Python Graph Gallery, and he replied with a link to _his source_: a data package used for teaching a course in Exploratory Data Analysis at the University of British Columbia. 

In [ ]:
%%html
<blockquote class="twitter-tweet" data-lang="en"><p lang="en" dir="ltr">Hi. Didn&#39;t receive your email... Gapminder comes from this R library: <a href="https://t.co/BU1IFIGSxm">https://t.co/BU1IFIGSxm</a>. I will add citation asap.</p>&mdash; R+Py Graph Galleries (@R_Graph_Gallery) <a href="https://twitter.com/R_Graph_Gallery/status/920074231269941248?ref_src=twsrc%5Etfw">October 16, 2017</a></blockquote> <script async src="//platform.twitter.com/widgets.js" charset="utf-8"></script>

Note one immediate outcome of our reaching out to the author of The Python Graph Gallery: he realized he was not citing the source of his data [3], and promised to add proper credit. _It's always good form to credit your sources!_

We visited the online repository of the data source, and posted an [issue report](https://github.com/jennybc/gapminder/issues/18) there, with our questions about data quality. The author promptly responded, saying that _her_ source was the [Gapminder.org website](http://www.gapminder.org/data/)—**Gapminder** is the non-profit founded by  Hans Rosling to host public data and visualizations. She also said: _" I don't doubt there could be data quality problems! It should definitely NOT be used as an authoritative source for life expectancy"_

So it turns out that the data we're using comes from a set of tools meant for teaching, and is not up-to-date with the latest vital statistics. The author ended up [adding a warning](https://github.com/jennybc/gapminder/commit/7b3ac7f477c78f21865fa7defea20e72cb9e2b8a) to make this clear to visitors of the repository on GitHub. 

#### This is a wonderful example of how people collaborate  online via the open-source model.

##### Note:

For the most accurate data, you can visit the website of the [World Bank](https://data.worldbank.org).

In [ ]:
for y in life_expect.year.unique():
    frame = life_expect[ life_expect.year == y ]
    minpop = frame['pop'].min()

In [ ]:
#url = 'http://docs.google.com/spreadsheet/pub?key=phAwcNAVuyj2tPLxKvvnNPA&output=xlsx'
#life_expect2 = pandas.read_excel(url)

In [ ]:
# drop the columns for years 1800 to 1949
#dropyears = list(range(1800,1950))
#life_expect2 = life_expect2.drop(dropyears, axis=1)

In [ ]:
#life_expect2.shape

In [ ]:
#lifeExp2_clean = life_expect2.dropna()

In [ ]:
#lifeExp2_clean.shape

## References

1. [The Soviet War in Afghanistan, 1979-1989](https://www.theatlantic.com/photo/2014/08/the-soviet-war-in-afghanistan-1979-1989/100786/), The Atlantic (2014), by Alan Taylor.

2. US National Research Council Roundtable on the Demography of Forced Migration; H.E. Reed, C.B. Keely, editors.  Forced Migration & Mortality (2001), National Academies Press, Washington DC; Chapter 5: The Demographic Analysis of Mortality Crises: The Case of Cambodia, 1970-1979, Patrick Heuveline. Available at: https://www.ncbi.nlm.nih.gov/books/NBK223346/

3. gapminder R data package. Licensed CC-BY 3.0 by Jennifer (Jenny) Bryan (2015)  https://github.com/jennybc/gapminder

In [ ]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../../style/custom.css'
HTML(open(css_file, "r").read())